In [1]:
import sys
import os
sys.path.append('/home/pranavsatheesh/host_galaxies/')

In [2]:
import illustris_python as il
import matplotlib.pyplot as plt
import h5py
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
import astropy.units as u
from astropy.cosmology import z_at_value

In [3]:
basePath = '/orange/lblecha/IllustrisTNG/Runs/TNG50-1/output'
merger_file_path = '/home/pranavsatheesh/host_galaxies/data/merger_files'

In [4]:
snap_list = np.arange(1,100)
redshifts = np.array([il.groupcat.loadHeader(basePath, snap)['Redshift'].item() for snap in snap_list])
scale_factors = np.array([il.groupcat.loadHeader(basePath, snap)['Time'].item() for snap in snap_list])
one_plus_z = 1.0 / scale_factors

In [5]:
z_values = one_plus_z - 1.0
print(snap_list[0],z_values[0])

def snap_to_z(snap):
    return one_plus_z[snap-1]-1

snap_to_z(1)

1 14.989173240042412


14.989173240042412

In [6]:
merger_file_1bh = merger_file_path+'/galaxy-mergers_TNG50-1_gas-100_dm-100_star-100_bh-001.hdf5'#merger file
merger_file_nobh = merger_file_path+'/galaxy-mergers_TNG50-1_gas-100_dm-100_star-100_bh-000.hdf5'

In [7]:
fmergers = h5py.File(merger_file_nobh, 'r')
fmergers.keys()

<KeysViewHDF5 ['InfallMassRatio', 'ProgMassRatio', 'ProgMassRatio_mod', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloGrNr', 'SubhaloHalfmassRadType', 'SubhaloLenType', 'SubhaloMassInHalfRadType', 'SubhaloMassInRadType', 'SubhaloMassType', 'SubhaloPos', 'SubhaloSFR', 'SubhaloVel', 'SubhaloVelDisp', 'fpMass', 'fpMass_mod', 'fpMasshistory', 'fpinfallMass', 'fpsnaphistory', 'npMass', 'npMass_mod', 'npMasshistory', 'npinfallMass', 'npsnaphistory', 'shids_subf', 'shids_tree', 'snaps', 'time']>

In [33]:
fmergers['ProgMassRatio_mod'][:]

array([0.13042818, 0.10740212, 0.0407346 , ..., 0.82631713, 0.82732099,
       0.26820847])

In [8]:
prog_mass_ratio = fmergers['ProgMassRatio_mod'][:]
prog_mass_ratio[prog_mass_ratio>1] = 1/prog_mass_ratio[prog_mass_ratio>1]

In [9]:
print("There are %d mergers and %d major mergers"%(len(prog_mass_ratio), np.sum(prog_mass_ratio>0.1)))

major_mergers_mask = prog_mass_ratio > 0.1

There are 7699 mergers and 5938 major mergers


In [37]:
snaps_galaxy_mergers = fmergers['snaps'][:,2]

In [39]:
merger_indices = np.where(snaps_galaxy_mergers == 60)[0]

In [40]:
major_mergers_mask[merger_indices]

array([ True,  True,  True,  True, False, False,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [44]:
merger_indices

array([ 867,  887,  895, 1210, 1430, 1880, 2182, 2784, 3082, 3090, 3477,
       3857, 4363, 4441, 4727, 4794, 5368, 5412, 5693, 6164, 6174, 6232,
       6270, 6327, 6488, 6518, 6783, 6833, 6870, 6886, 6912, 6951, 7048,
       7051, 7318, 7390, 7481, 7489, 7512])

In [43]:
merger_indices[major_mergers_mask[merger_indices]]

array([ 867,  887,  895, 1210, 2182, 2784, 3090, 3477, 3857, 4363, 4441,
       4727, 4794, 5368, 5693, 6164, 6174, 6232, 6270, 6327, 6488, 6518,
       6783, 6870, 6912, 6951, 7048, 7051, 7318, 7390, 7481, 7489, 7512])

In [11]:
descendant_subfind_id = fmergers['shids_subf'][major_mergers_mask][0][2]
descendant_snap = fmergers['snaps'][major_mergers_mask][0][2]
print("Descendant subfind ID: %d, snap: %d"%(descendant_subfind_id, descendant_snap))

Descendant subfind ID: 4, snap: 61


In [12]:
tree = il.sublink.loadTree(basePath,snapNum=descendant_snap,id=descendant_subfind_id,fields=['SubhaloID', 'NextProgenitorID', 'MainLeafProgenitorID', 'FirstProgenitorID',
              'LastProgenitorID', 'RootDescendantID', 'SubhaloLenType', 'SubhaloMassType',
              'SnapNum', 'DescendantID', 'SubfindID','FirstSubhaloInFOFGroupID']
,onlyMDB=True)

In [13]:
rootID = tree['SubhaloID'][-1]
descID = tree['DescendantID'][-1]
descSnap = tree['SnapNum'][-1]


desc_time = cosmo.age(snap_to_z(descSnap)).value #in Gyr
#find the redshift of descendant 2 Gyrs after

target_descendant_z = z_at_value(cosmo.age, (desc_time+2) * u.Gyr).value
target_descendant_snap = snap_list[np.argmin(np.abs(z_values - target_descendant_z))]

print(target_descendant_snap)

74


In [14]:
merger_descendants = {
    'shids_subfind': [],
    'snap': [],
    'shids_tree': []
}

In [15]:
rootID = tree['SubhaloID'][-1]
descID = tree['DescendantID'][-1]
descSnap = tree['SnapNum'][-1]
desc_subfind_id = tree['SubfindID'][-1]

print("Descendant ID: %d, snap: %d, subhaloid: %d"%(descID, descSnap,desc_subfind_id))

while descID != -1:
    index = - (1 + rootID-descID)
    subID = tree['SubhaloID'][index]
    descID = tree['DescendantID'][index]
    descSnap = tree['SnapNum'][index]
    desc_subfind_id = tree['SubfindID'][index]
    #find the descendant 
    if(descSnap> target_descendant_snap):
        break
    else:
        print("Descendant ID: %d, snap: %d, subhaloid: %d"%(descID, descSnap,desc_subfind_id))
        merger_descendants['shids_subfind'].append(desc_subfind_id)
        merger_descendants['snap'].append(descSnap)
        merger_descendants['shids_tree'].append(subID)




Descendant ID: 99118377, snap: 61, subhaloid: 4
Descendant ID: 99118376, snap: 62, subhaloid: 3
Descendant ID: 99118375, snap: 63, subhaloid: 3
Descendant ID: 99118374, snap: 64, subhaloid: 5
Descendant ID: 99118373, snap: 65, subhaloid: 5
Descendant ID: 99118372, snap: 66, subhaloid: 7
Descendant ID: 99118371, snap: 67, subhaloid: 5
Descendant ID: 99118370, snap: 68, subhaloid: 4
Descendant ID: 99118369, snap: 69, subhaloid: 5
Descendant ID: 99118368, snap: 70, subhaloid: 4
Descendant ID: 99118367, snap: 71, subhaloid: 5
Descendant ID: 99118366, snap: 72, subhaloid: 6
Descendant ID: 99118365, snap: 73, subhaloid: 6
Descendant ID: 99118364, snap: 74, subhaloid: 6


## Working with the major merger descendant file to filter out in the population

In [16]:
descendant_file = merger_file_path+'/descendants_after_2Gyr_of_mergers.hdf5'
fdescendant = h5py.File(descendant_file, 'r')
fdescendant.keys()

<KeysViewHDF5 ['shids_subfind', 'shids_tree', 'snap']>

In [17]:
snaps = fdescendant['snap'][:]
shids_subfind = fdescendant['shids_subfind'][:]

pairs = np.array(list(zip(snaps, shids_subfind)))
unique_pairs, counts = np.unique(pairs, axis=0, return_counts=True)

repeated_pairs = unique_pairs[counts > 1]
print("Number of repeated pairs:", len(repeated_pairs))


Number of repeated pairs: 11060


In [18]:
snap_list = np.arange(1,100)

In [19]:
subhalos = il.groupcat.loadSubhalos(
                basePath, snap_list[-1], 
                fields=['SubhaloLenType', 'SubhaloMassType', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloSFR','SubhaloGasMetallicity','SubhaloStarMetallicity','SubhaloPos','SubhaloHalfmassRadType']
            )

In [20]:
descend_snaps = np.where(snaps==snap_list[-1])[0]
descend_subhaloids = shids_subfind[descend_snaps]

In [21]:
subhalo_ids = np.arange(len(subhalos['SubhaloLenType'][:,5]))

In [31]:
descendant_mask = ~np.isin(subhalo_ids, descend_subhaloids)

In [32]:
subhalo_ids[descendant_mask]

array([      1,       2,       3, ..., 5688110, 5688111, 5688112])

In [22]:
np.setdiff1d(subhalo_ids, descend_subhaloids)

array([      1,       2,       3, ..., 5688110, 5688111, 5688112])

In [23]:
len(descend_subhaloids)

163

In [24]:
len(subhalo_ids)

5688113

In [25]:
subhalo_ids[descend_subhaloids]

array([     0,  63877,  63889,  63980, 117255, 117255, 143882, 229933,
       229933, 253861, 294873, 300903, 300903, 329512, 371126, 382215,
       383976, 383980, 418335, 419618, 419626, 430866, 434356, 438148,
       440410, 441709, 441709, 449658, 455292, 470346, 474008, 478219,
       486046, 493433, 499021, 499704, 500579, 502372, 509709, 527839,
       539667, 561004, 563050, 568303, 569653, 579509, 581058, 583258,
       590569, 592984, 594731, 600743, 600743, 602133, 606038, 607864,
       608061, 608528, 611524, 614626, 614627, 614627, 617027, 624089,
       629093, 630290, 631927, 637317, 637317, 639157, 641601, 642734,
       643728, 645213, 645833, 647894, 648911, 648986, 649207, 649905,
       652044, 655988, 656798, 659936, 660178, 661934, 669749, 673482,
       673482, 674870, 674870, 677069, 678287, 678357, 678357, 678672,
       681133, 681133, 685739, 687331, 692122, 694385, 694760, 694832,
       695819, 697626, 700600, 703534, 704456, 712942, 717408, 718908,
      